<a href="https://colab.research.google.com/github/FullPowerMore02/data-mining/blob/main/project08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests
!pip install newspaper3k
!pip install pandas
!pip install openpyxl
!pip install tabulate
!pip install bs4
!pip install urllib3

In [ ]:
# เรียกใช้โมดูลที่จำเป็น
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen
from datetime import datetime, timedelta
from openpyxl import Workbook
import pandas as pd
import newspaper
import numpy as np

In [ ]:
#ดึงข้อมูล
# ฟังก์ชันสำหรับดึงเนื้อหาของบทความจากลิงก์ที่ระบุ
def get_article_content(link):
    article_client = urlopen(link)
    article_page = soup(article_client, "html.parser")
    article_content = article_page.find("div", {"class": "article-body"})

    if article_content:
        return article_content.text.strip()
    else:
        return "No article content found."

# ฟังก์ชันสำหรับกรองข้อมูลข่าวที่ตรงกับคำค้นหาที่กำหนด
def filter_news(news_list, search_keywords):
    filtered_news = []
    for news in news_list:
        title = news.title.text
        if any(keyword in title.lower() for keyword in search_keywords):
            filtered_news.append(news)
    return filtered_news

# ฟังก์ชันหลักที่ใช้ในการดึงข้อมูลข่าว
def news():
    search_keywords = ["cryptocurrency", "bitcoin","btc"]
    url_keywords = "+OR+".join(search_keywords)

    # กำหนดวันที่เริ่มต้นและวันสิ้นสุดที่ต้องการดึงข้อมูล
    start_date = datetime(2022, 1, 10)
    end_date = datetime(2022, 1, 12)

    current_date = start_date

    # สร้าง Workbook เพื่อเก็บข้อมูลที่ได้รับ
    workbook = Workbook()
    sheet = workbook.active
    sheet.append(["Title", "Link", "Publication Date", "Article Content"])

    # ดึงข้อมูลข่าวในช่วงวันที่ระบุ
    while current_date <= end_date:
        search_date = current_date.strftime("%Y-%m-%d")

        url = f"https://news.google.com/rss/search?q={search_date}%20{url_keywords}&hl=en-US&gl=US&ceid=US%3Aen"

        # ดึง XML จาก Google News และแปลงให้อยู่ในรูปแบบ XML ด้วย BeautifulSoup
        client = urlopen(url)
        xml_page = client.read()
        client.close()

        page = soup(xml_page, "xml")
        news_list = page.findAll("item")

        # กรองข่าวตามคำค้นหา
        news_list = filter_news(news_list, search_keywords)

        # ดึงเนื้อหาของบทความและบันทึกข้อมูลลงใน sheet ของ Workbook
        for news in news_list:
            title = news.title.text
            link = news.link.text

            article_content = get_article_content(link)

            sheet.append([title, link, current_date.strftime("%Y-%m-%d"), article_content])

        current_date += timedelta(days=1)

    # บันทึกไฟล์ Excel
    workbook.save("news_data02.xlsx")

# เรียกใช้ฟังก์ชันหลักในกรณีที่โค้ดถูกเรียกใช้งานโดยตรงจากไฟล์
if __name__ == "__main__":
    news()


In [ ]:
df = pd.read_excel('/content/news_data02.xlsx')
df


,Title,Link,Publication Date,Article Content
0,Como Vender Bitcoins En Binance - Fantom Offic...,https://news.google.com/rss/articles/CBMiKWh0d...,2022-01-10,No article content found.
1,Bitcoin Correction - Are Etfs A Safe Investmen...,https://news.google.com/rss/articles/CBMiJmh0d...,2022-01-10,No article content found.
2,Extremists Receive Large Ethereum Cryptocurren...,https://news.google.com/rss/articles/CBMiaWh0d...,2022-01-10,No article content found.
3,Crypto NFT Today: The Latest News in Blockchai...,https://news.google.com/rss/articles/CBMib2h0d...,2022-01-10,No article content found.
4,Bitcoin Price Dollars Away From First Ever Wee...,https://news.google.com/rss/articles/CBMiQWh0d...,2022-01-10,No article content found.
...,...,...,...,...
93,Top NFT Projects for Investors in 2023 - Bitco...,https://news.google.com/rss/articles/CBMiNmh0d...,2022-01-12,No article content found.
94,Investors Seek Criminal Charges After $3.6B Bi...,https://news.google.com/rss/articles/CBMiX2h0d...,2022-01-12,No article content found.
95,Profiting On Biogas With Bitcoin Mining - Bitc...,https://news.google.com/rss/articles/CBMiTGh0d...,2022-01-12,No article content found.
96,More Green Energy: Crypto Mining Saves A Hydro...,https://news.google.com/rss/articles/CBMibWh0d...,2022-01-12,No article content found.


In [ ]:
#หาเนื้อข่าว
def get_destination_link(link):
    try:
        page_client = urlopen(link)
        page_content = page_client.read().decode('utf-8')
        page_soup = soup(page_content, 'html.parser')
        destination_link = page_soup.find('a')['href']
        return destination_link
    except:
        return np.nan

# อ่านไฟล์ Excel
df_links = pd.read_excel("/content/news_data02.xlsx")

# สร้างรายการเพื่อเก็บผลลัพธ์ของบทความ
results = []

# ใช้คอลัมน์ "Link" เพื่อดึง Destination Link และเนื้อหาของบทความ
for index, row in df_links.iterrows():
    link = row["Link"]
    destination_link = get_destination_link(link)

    print("Destination Link:", destination_link)

    try:
        article = newspaper.Article(link)
        article.download()
        article.parse()

        title = article.title
        news_content = article.text
        authors = article.authors
        publish_date = article.publish_date
        top_image = article.top_image

        print("Title:", title)
        print("News Content:", news_content)
        print("Authors:", authors)
        print("Publish Date:", publish_date)
        print("Top Image URL:", top_image)

        # แปลงวันที่และเวลาให้เป็นข้อมูลปกติ (timezone unaware)
        if publish_date is not None:
            publish_date = publish_date.replace(tzinfo=None)

        # เพิ่มผลลัพธ์ของบทความลงในรายการ
        results.append({
            "Destination Link": destination_link,
            "Title": title,
            "News Content": news_content,
            "Authors": authors,
            "Publish Date": publish_date,
            "Top Image URL": top_image
        })

    except Exception as e:
        print("Error occurred:", str(e))
        print("Skipping to next link...")
        # เพิ่มแถวผลลัพธ์ที่มีค่า NaN ในกรณีที่การดึงข้อมูลไม่สำเร็จ
        results.append({
            "Destination Link": np.nan,
            "Title": np.nan,
            "News Content": np.nan,
            "Authors": np.nan,
            "Publish Date": np.nan,
            "Top Image URL": np.nan
        })

    print("-----------------------")

# สร้าง DataFrame จากผลลัพธ์ของบทความ
df_results = pd.DataFrame(results)

# บันทึกลงในไฟล์ Excel
df_results.to_excel("article_results.xlsx", index=False)


Destination Link: https://www.indieshuffle.com/eth/LW8LA62b
Error occurred: Article `download()` failed with 404 Client Error: Not Found for url: https://www.indieshuffle.com/eth/LW8LA62b on URL https://news.google.com/rss/articles/CBMiKWh0dHBzOi8vd3d3LmluZGllc2h1ZmZsZS5jb20vZXRoL0xXOExBNjJi0gEA?oc=5
Skipping to next link...
-----------------------
Destination Link: https://www.indieshuffle.com/?nVxSKkA4
Title: Indie Shuffle Music Blog :: Discover New Songs And Playlists
News Content: When the current song has ended you'll see it here
Authors: []
Publish Date: None
Top Image URL: https://i1.sndcdn.com/artworks-4SGx1mbBh6qB-0-t500x500.jpg
-----------------------
Destination Link: https://www.splcenter.org/hatewatch/2023/05/10/extremists-receive-large-ethereum-cryptocurrency-donations
Title: Extremists Receive Large Ethereum Cryptocurrency Donations
News Content: The single biggest beneficiary was antigovernment extremist Ammon Bundy’s People’s Rights Network (PRN), which received transf

In [ ]:
df = pd.read_excel('/content/article_results.xlsx')
df


,Destination Link,Title,News Content,Authors,Publish Date,Top Image URL
0,NaN,NaN,NaN,NaN,NaT,NaN
1,https://www.indieshuffle.com/?nVxSKkA4,Indie Shuffle Music Blog :: Discover New Songs...,When the current song has ended you'll see it ...,[],NaT,https://i1.sndcdn.com/artworks-4SGx1mbBh6qB-0-...
2,https://www.splcenter.org/hatewatch/2023/05/10...,Extremists Receive Large Ethereum Cryptocurren...,The single biggest beneficiary was antigovernm...,[],NaT,https://www.splcenter.org/sites/default/files/...
3,https://innotechtoday.com/crypto-nft-today-the...,Crypto NFT Today: The Latest News in Blockchai...,Welcome to another edition of Crypto NFT Today...,"['I T Today', 'All Posts', 'Var Params', 'Clea...",2023-01-04 17:32:28,https://innotechtoday.com/wp-content/uploads/2...
4,https://blockworks.co/news/bitcoin-price-first...,Bitcoin Price Dollars Away From First Ever Wee...,Bitcoin (BTC) is painfully close to recording ...,['Sebastian Sinclair'],2023-02-08 06:38:20,https://blockworks.co/_next/image?url=https://...
...,...,...,...,...,...,...
93,https://www.bitcoinmarketjournal.com/top-nft-p...,Top NFT Projects for Investors in 2023,Summary: NFTs are a specialized niche of crypt...,['Anatol Antonovici'],2023-01-05 08:00:35,https://www.bitcoinmarketjournal.com/wp-conten...
94,https://decrypt.co/90274/investors-seek-crimin...,Investors Seek Criminal Charges After $3.6B Bi...,"Raees and Ameer Cajee, the brothers behind cry...","['Decrypt', 'Scott Chipolina']",2022-01-12 13:21:00,https://cdn.decrypt.co/resize/1024/height/512/...
95,NaN,NaN,NaN,NaN,NaT,NaN
96,https://www.newsbtc.com/news/bitcoin/more-gree...,More Green Energy: Crypto Mining Saves A Hydro...,Green energy powers most of Bitcoin mining and...,"['Eduardo Próspero', 'Eduardo Prospero Is An A...",2022-01-12 14:11:05,https://www.newsbtc.com/wp-content/uploads/202...


In [ ]:
#รวมไฟล์
# อ่านไฟล์ Excel news_data02.xlsx
df_news = pd.read_excel("news_data02.xlsx")

# อ่านไฟล์ Excel article_results.xlsx
df_results = pd.read_excel("article_results.xlsx")

# สร้าง DataFrame ใหม่ที่รวมคอลัมน์ที่ต้องการจากทั้งสอง DataFrame
df_combined = pd.concat([df_news["Title"], df_news["Publication Date"], df_results["Destination Link"], df_results["News Content"]], axis=1)

# บันทึกลงในไฟล์ Excel
df_combined.to_excel("combined_results.xlsx", index=False)


In [ ]:
df = pd.read_excel('/content/combined_results.xlsx')
df


,Title,Publication Date,Destination Link,News Content
0,Como Vender Bitcoins En Binance - Fantom Offic...,2022-01-10,NaN,NaN
1,Bitcoin Correction - Are Etfs A Safe Investmen...,2022-01-10,https://www.indieshuffle.com/?nVxSKkA4,When the current song has ended you'll see it ...
2,Extremists Receive Large Ethereum Cryptocurren...,2022-01-10,https://www.splcenter.org/hatewatch/2023/05/10...,The single biggest beneficiary was antigovernm...
3,Crypto NFT Today: The Latest News in Blockchai...,2022-01-10,https://innotechtoday.com/crypto-nft-today-the...,Welcome to another edition of Crypto NFT Today...
4,Bitcoin Price Dollars Away From First Ever Wee...,2022-01-10,https://blockworks.co/news/bitcoin-price-first...,Bitcoin (BTC) is painfully close to recording ...
...,...,...,...,...
93,Top NFT Projects for Investors in 2023 - Bitco...,2022-01-12,https://www.bitcoinmarketjournal.com/top-nft-p...,Summary: NFTs are a specialized niche of crypt...
94,Investors Seek Criminal Charges After $3.6B Bi...,2022-01-12,https://decrypt.co/90274/investors-seek-crimin...,"Raees and Ameer Cajee, the brothers behind cry..."
95,Profiting On Biogas With Bitcoin Mining - Bitc...,2022-01-12,NaN,NaN
96,More Green Energy: Crypto Mining Saves A Hydro...,2022-01-12,https://www.newsbtc.com/news/bitcoin/more-gree...,Green energy powers most of Bitcoin mining and...


In [ ]:
#หาส่วนซ้ำของข่าว
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# โหลดข้อมูลจากไฟล์ Excel ใน DataFrame
df = pd.read_excel('/content/combined_results.xlsx')

# เลือกเฉพาะคอลัมน์ "News Content" เพื่อใช้ในการหาความคล้ายคลึง
selected_columns = df["News Content"]

# แทนที่ค่า NaN ด้วยข้อความว่างเพื่อประมวลผลทั้ง DataFrame
selected_columns.fillna("", inplace=True)

# สร้าง TfidfVectorizer เพื่อแปลงข้อความเป็นเวกเตอร์ TF-IDF
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(selected_columns)

# หาความคล้ายคลึงระหว่างข้อความทุกคู่โดยใช้ cosine similarity
similarity_matrix = cosine_similarity(tfidf_matrix)

# สร้าง DataFrame ใหม่เพื่อเก็บความคล้ายคลึงระหว่างข้อความ
similarity_df = pd.DataFrame(similarity_matrix, columns=selected_columns.index, index=selected_columns.index)

# แสดงผล DataFrame ความคล้ายคลึง
print(similarity_df)

# บันทึก DataFrame ความคล้ายคลึงเป็นไฟล์ Excel
output_file = '/content/similarity_results.xlsx'
similarity_df.to_excel(output_file, index=False)
print(f"Similarity results saved to {output_file}")


     0         1         2         3         4    5         6    7         8   \
0   0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.000000   
1   0.0  1.000000  0.054294  0.077725  0.056646  0.0  0.123252  0.0  0.027897   
2   0.0  0.054294  1.000000  0.262000  0.192934  0.0  0.310425  0.0  0.136469   
3   0.0  0.077725  0.262000  1.000000  0.289408  0.0  0.488488  0.0  0.135389   
4   0.0  0.056646  0.192934  0.289408  1.000000  0.0  0.338838  0.0  0.103532   
..  ...       ...       ...       ...       ...  ...       ...  ...       ...   
93  0.0  0.074917  0.240845  0.420030  0.223641  0.0  0.380094  0.0  0.115487   
94  0.0  0.072493  0.238005  0.301674  0.205245  0.0  0.368178  0.0  0.105460   
95  0.0  0.000000  0.000000  0.000000  0.000000  0.0  0.000000  0.0  0.000000   
96  0.0  0.074803  0.223806  0.332668  0.229127  0.0  0.415754  0.0  0.134269   
97  0.0  0.082770  0.288033  0.484101  0.276453  0.0  0.475763  0.0  0.140458   

          9   ...        88

In [ ]:
df = pd.read_excel('/content/similarity_results.xlsx')
df


,0,1,2,3,4,5,6,7,8,9,...,88,89,90,91,92,93,94,95,96,97
0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,...,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000
1,0,1.000000,0.054294,0.077725,0.056646,0,0.123252,0,0.027897,0.052505,...,0.104147,0,0,0.095968,0.078612,0.074917,0.072493,0,0.074803,0.082770
2,0,0.054294,1.000000,0.262000,0.192934,0,0.310425,0,0.136469,0.202390,...,0.294201,0,0,0.199812,0.224739,0.240845,0.238005,0,0.223806,0.288033
3,0,0.077725,0.262000,1.000000,0.289408,0,0.488488,0,0.135389,0.280725,...,0.393763,0,0,0.318544,0.333759,0.420030,0.301674,0,0.332668,0.484101
4,0,0.056646,0.192934,0.289408,1.000000,0,0.338838,0,0.103532,0.221009,...,0.294058,0,0,0.215017,0.222134,0.223641,0.205245,0,0.229127,0.276453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,0,0.074917,0.240845,0.420030,0.223641,0,0.380094,0,0.115487,0.240309,...,0.335103,0,0,0.265694,0.280421,1.000000,0.249345,0,0.284037,0.412717
94,0,0.072493,0.238005,0.301674,0.205245,0,0.368178,0,0.105460,0.226480,...,0.342308,0,0,0.248748,0.263708,0.249345,1.000000,0,0.271575,0.307446
95,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,0,0.000000,0.000000,...,0.000000,0,0,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000
96,0,0.074803,0.223806,0.332668,0.229127,0,0.415754,0,0.134269,0.265870,...,0.378574,0,0,0.384940,0.297226,0.284037,0.271575,0,1.000000,0.347706


In [ ]:
!pip install vaderSentiment


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 2.5 MB/s eta 0:00:00


In [ ]:
#ใช้news content
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# อ่านไฟล์ Excel news_data02.xlsx
df_news = pd.read_excel("news_data02.xlsx")

# อ่านไฟล์ Excel article_results.xlsx
df_results = pd.read_excel("article_results.xlsx")

# สร้าง DataFrame ใหม่ที่รวมคอลัมน์ที่ต้องการจากทั้งสอง DataFrame
df_combined = pd.concat([df_news["Title"], df_news["Publication Date"], df_results["Destination Link"], df_results["News Content"]], axis=1)

# แทนค่า NaN ด้วยข้อความว่างในคอลัมน์ 'News Content'
df_combined['News Content'] = df_combined['News Content'].fillna('')

# สร้างฟังก์ชันสำหรับการตรวจจับอารมณ์ในข้อความ
def get_sentiment_score(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# สร้างวัวเครื่องคำนวณ Sentiment
analyzer = SentimentIntensityAnalyzer()

df_combined['Sentiment Score'] = df_combined['News Content'].apply(get_sentiment_score)

# บันทึกลงในไฟล์ Excel
df_combined.to_excel("combined_results_with_sentiment.xlsx", index=False)


In [ ]:
df = pd.read_excel('/content/combined_results_with_sentiment.xlsx')
df

,Title,Publication Date,Destination Link,News Content,Sentiment Score
0,Como Vender Bitcoins En Binance - Fantom Offic...,2022-01-10,NaN,NaN,0.0000
1,Bitcoin Correction - Are Etfs A Safe Investmen...,2022-01-10,https://www.indieshuffle.com/?nVxSKkA4,When the current song has ended you'll see it ...,0.0000
2,Extremists Receive Large Ethereum Cryptocurren...,2022-01-10,https://www.splcenter.org/hatewatch/2023/05/10...,The single biggest beneficiary was antigovernm...,0.8604
3,Crypto NFT Today: The Latest News in Blockchai...,2022-01-10,https://innotechtoday.com/crypto-nft-today-the...,Welcome to another edition of Crypto NFT Today...,0.9705
4,Bitcoin Price Dollars Away From First Ever Wee...,2022-01-10,https://blockworks.co/news/bitcoin-price-first...,Bitcoin (BTC) is painfully close to recording ...,-0.9592
...,...,...,...,...,...
93,Top NFT Projects for Investors in 2023 - Bitco...,2022-01-12,https://www.bitcoinmarketjournal.com/top-nft-p...,Summary: NFTs are a specialized niche of crypt...,0.9998
94,Investors Seek Criminal Charges After $3.6B Bi...,2022-01-12,https://decrypt.co/90274/investors-seek-crimin...,"Raees and Ameer Cajee, the brothers behind cry...",-0.9595
95,Profiting On Biogas With Bitcoin Mining - Bitc...,2022-01-12,NaN,NaN,0.0000
96,More Green Energy: Crypto Mining Saves A Hydro...,2022-01-12,https://www.newsbtc.com/news/bitcoin/more-gree...,Green energy powers most of Bitcoin mining and...,0.9980


In [ ]:
#ใช้แต่ title
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

# อ่านไฟล์ Excel news_data02.xlsx
df_news = pd.read_excel("news_data02.xlsx")

# สร้างวัวเครื่องคำนวณ Sentiment
analyzer = SentimentIntensityAnalyzer()

# สร้างฟังก์ชันสำหรับการคำนวณคะแนนความรู้สึกในข้อความในคอลัมน์ 'Title'
def get_sentiment_score(text):
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']

# คำนวณคะแนนความรู้สึกในคอลัมน์ 'Title'
df_news['Sentiment Score'] = df_news['Title'].apply(get_sentiment_score)

# บันทึกลงในไฟล์ Excel
df_news.to_excel("news_data_with_sentiment.xlsx", index=False)


In [ ]:
df = pd.read_excel('/content/combined_results_with_sentiment.xlsx')
df

,Title,Publication Date,Destination Link,News Content,Sentiment Score
0,Como Vender Bitcoins En Binance - Fantom Offic...,2022-01-10,NaN,NaN,0.0000
1,Bitcoin Correction - Are Etfs A Safe Investmen...,2022-01-10,https://www.indieshuffle.com/?nVxSKkA4,When the current song has ended you'll see it ...,0.0000
2,Extremists Receive Large Ethereum Cryptocurren...,2022-01-10,https://www.splcenter.org/hatewatch/2023/05/10...,The single biggest beneficiary was antigovernm...,0.8604
3,Crypto NFT Today: The Latest News in Blockchai...,2022-01-10,https://innotechtoday.com/crypto-nft-today-the...,Welcome to another edition of Crypto NFT Today...,0.9705
4,Bitcoin Price Dollars Away From First Ever Wee...,2022-01-10,https://blockworks.co/news/bitcoin-price-first...,Bitcoin (BTC) is painfully close to recording ...,-0.9592
...,...,...,...,...,...
93,Top NFT Projects for Investors in 2023 - Bitco...,2022-01-12,https://www.bitcoinmarketjournal.com/top-nft-p...,Summary: NFTs are a specialized niche of crypt...,0.9998
94,Investors Seek Criminal Charges After $3.6B Bi...,2022-01-12,https://decrypt.co/90274/investors-seek-crimin...,"Raees and Ameer Cajee, the brothers behind cry...",-0.9595
95,Profiting On Biogas With Bitcoin Mining - Bitc...,2022-01-12,NaN,NaN,0.0000
96,More Green Energy: Crypto Mining Saves A Hydro...,2022-01-12,https://www.newsbtc.com/news/bitcoin/more-gree...,Green energy powers most of Bitcoin mining and...,0.9980


ช่วงทดสอบ NPL ต่างๆ

In [ ]:
#Naive Bayes Classifier
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

# ดาวน์โหลดข้อมูลสำหรับ Vader Sentiment Analyzer และ punkt tokenizer
nltk.download('vader_lexicon')
nltk.download('punkt')

# อ่านไฟล์ Excel ที่มีคอลัมน์ 'Title' และ 'Sentiment Score'
df = pd.read_excel('/content/combined_results_with_sentiment.xlsx')

# สร้างตัวเครื่องคำนวณความรู้สึกด้วย Vader Sentiment Analyzer จาก `nltk`
analyzer = SentimentIntensityAnalyzer()

# ฟังก์ชันสำหรับคำนวณความรู้สึกในข้อความ
def get_sentiment_score(text):
    tokens = nltk.word_tokenize(text.lower())
    words = dict([(word, True) for word in tokens])
    sentiment = analyzer.polarity_scores(text)['compound']
    return sentiment

# คำนวณความรู้สึกในข้อความในคอลัมน์ 'Title'
df['Sentiment Score'] = df['Title'].apply(get_sentiment_score)

# บันทึกลงในไฟล์ Excel
df.to_excel("combined_results_with_sentiment_score.xlsx", index=False)


[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
df = pd.read_excel('/content/combined_results_with_sentiment_score.xlsx')
df

,Title,Publication Date,Destination Link,News Content,Sentiment Score
0,Como Vender Bitcoins En Binance - Fantom Offic...,2022-01-10,NaN,NaN,0.0000
1,Bitcoin Correction - Are Etfs A Safe Investmen...,2022-01-10,https://www.indieshuffle.com/?nVxSKkA4,When the current song has ended you'll see it ...,0.4404
2,Extremists Receive Large Ethereum Cryptocurren...,2022-01-10,https://www.splcenter.org/hatewatch/2023/05/10...,The single biggest beneficiary was antigovernm...,-0.5106
3,Crypto NFT Today: The Latest News in Blockchai...,2022-01-10,https://innotechtoday.com/crypto-nft-today-the...,Welcome to another edition of Crypto NFT Today...,0.3818
4,Bitcoin Price Dollars Away From First Ever Wee...,2022-01-10,https://blockworks.co/news/bitcoin-price-first...,Bitcoin (BTC) is painfully close to recording ...,-0.5994
...,...,...,...,...,...
93,Top NFT Projects for Investors in 2023 - Bitco...,2022-01-12,https://www.bitcoinmarketjournal.com/top-nft-p...,Summary: NFTs are a specialized niche of crypt...,0.2023
94,Investors Seek Criminal Charges After $3.6B Bi...,2022-01-12,https://decrypt.co/90274/investors-seek-crimin...,"Raees and Ameer Cajee, the brothers behind cry...",-0.6705
95,Profiting On Biogas With Bitcoin Mining - Bitc...,2022-01-12,NaN,NaN,0.3818
96,More Green Energy: Crypto Mining Saves A Hydro...,2022-01-12,https://www.newsbtc.com/news/bitcoin/more-gree...,Green energy powers most of Bitcoin mining and...,0.3353


In [ ]:
import pandas as pd
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import CountVectorizer

# อ่านไฟล์ Excel ที่มีคอลัมน์ 'Title' และ 'Sentiment Score'
df = pd.read_excel('/content/combined_results_with_sentiment.xlsx')

# สร้างวัวเครื่องคำนวณความรู้สึกด้วย Vader Sentiment Analyzer จาก `nltk`
analyzer = SentimentIntensityAnalyzer()

# ฟังก์ชันสำหรับคำนวณความรู้สึกในข้อความ
def get_sentiment_score(text):
    tokens = nltk.word_tokenize(text.lower())
    words = dict([(word, True) for word in tokens])
    sentiment = analyzer.polarity_scores(text)['compound']
    return sentiment

# คำนวณความรู้สึกในข้อความในคอลัมน์ 'Title'
df['Sentiment Score'] = df['Title'].apply(get_sentiment_score)

# แบ่งข้อมูลเป็นชุดฝึกสอนและชุดทดสอบ
X_train, X_test, y_train, y_test = train_test_split(df['Title'], df['Sentiment Score'], test_size=0.1, random_state=42)

# สร้างตัวแปรเก็บคำศัพท์ (vocabulary) และเตรียมข้อมูลให้สามารถนำไปฝึกสอน
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)

# สร้างและฝึกสอนโมเดล Linear Regression
classifier = LinearRegression()
classifier.fit(X_train_vectorized, y_train)

# ทำนายความรู้สึกในข้อความในชุดทดสอบ
y_pred = classifier.predict(X_test_vectorized)

# แสดงผลลัพธ์
result_df = pd.DataFrame({'Title': X_test, 'True Sentiment Score': y_test, 'Predicted Sentiment Score': y_pred})

# บันทึกลงในไฟล์ Excel
result_df.to_excel("sentiment_prediction_results.xlsx", index=False)


In [ ]:
df = pd.read_excel('/content/sentiment_prediction_results.xlsx')
df

,Title,True Sentiment Score,Predicted Sentiment Score
0,Will Bitcoin Be Done In By Terrible Zoning Law...,-0.4767,-1.864207e-02
1,HIVE Blockchain Announces Rebranding and Name ...,0.0000,3.143303e-01
2,Investors Seek Criminal Charges After $3.6B Bi...,-0.6705,-4.663737e-02
3,Developer will soon accept cryptocurrency as p...,0.3818,-1.213001e-01
4,Tiny Bitcoin miner defies massive odds to solv...,-0.2732,-1.937241e-01
5,How cryptocurrency investing is evolving to at...,0.3612,3.611972e-01
6,Tesla owners reveal how they mine for Bitcoin ...,0.0000,1.367657e-07
7,Ustn Coin - Buy Cspr Crypto-trace bitcoin wall...,0.0000,-3.159509e-02
8,"Bitcoin briefly tumbles below $40,000 to the l...",-0.5719,-6.328716e-02
9,Como Vender Bitcoins En Binance - Fantom Offic...,0.0000,1.439188e-01
